In [1]:
using Pkg 
Pkg.add("NamedArrays")
Pkg.build("NamedArrays")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using JuMP, Cbc
properties = [:height,:flowers,:goodsmell,:beauty,:goodpollinator]
plants = [:p1,:p2,:p3,:p4,:p5,:p6]
cost_per_plant = Dict(zip(plants,[5.5,7.3,7.8,6.3,5.0,7.9]))

property_lower_bound = Dict(zip(properties,[315,500,450,400,300]))

using NamedArrays
# array containing amount of property (row) in each plant (column)
prop_per_plant = NamedArray([3.1 4 1.5 1.8 3.3 5
4.3 4.3 1.3 4 1.1 1.7
3.6 1.8 4.6 2.2 1.5 0.7
4 1.6 4.6 4 4 2.8
3.6 4.9 2.6 1.9 1.3 2.4],
(properties,plants),("property","plant"))

# max number of each type of plant is 100
max_buy = 100;

m = Model()

#Number of units purchased for plant p 
@variable(m,0 <= x[plants] <= max_buy)

#Objective - Minimize the cost of planting the garden, cost per plant * number of plants 
@objective(m,Min, sum(cost_per_plant[p]x[p] for p in plants))

#constraints 
@constraint(m, minreqc[i in properties],
    sum(x[p]prop_per_plant[i,p] for p in plants) >= property_lower_bound[i])

set_optimizer(m, Cbc.Optimizer)
optimize!(m)

#print total cost 
println("Minimum cost of planting this garden: \$", objective_value(m))

println("Number of plants purchased: ")
for p in plants 
    println(p, ": ", value(x[p]))
end



Minimum cost of planting this garden: $736.5057915057914
Number of plants purchased: 
p1: 100.0
p2: 0.0
p3: 13.256113256113256
p4: 13.191763191763183
p5: 0.0
p6: 0.0
Presolve 5 (0) rows, 6 (0) columns and 30 (0) elements
0  Obj 0 Primal inf 425.28617 (5)
2  Obj 736.50579
Optimal - objective value 736.50579
Optimal objective 736.5057915 - 2 iterations time 0.002
